# Learning XGBoost model

In [1]:
%reset 
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import xgboost as xgb
from numpy import nan as NA
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score as auc

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


# Load from accident data and predict on it

Load the accident data from disk, feed into a xgboost model

In [2]:
nbatch = 10000

### Read files and split for validation

In [3]:
label_file = pd.read_csv("./train/labels_ext.csv", index_col=0)
y = label_file['DRUNK_DR'].apply(lambda x: x*1 ).values

In [4]:
type(y)

numpy.ndarray

In [5]:
Xtrain = pd.read_csv("./train/accident_train_ext.csv", index_col=0)

In [6]:
n_train, n_dim = Xtrain.shape

Drop ID

In [7]:
Xtrain.drop('ID', axis= 1, inplace=True)
print("Splitting data")

Splitting data


In [8]:
Xtrain_c, Xeval, Ytrain_c, Yeval = train_test_split(Xtrain.values, y, test_size = 0.3)

### Load data into sparse matrix 

In [9]:
print("Load data into sparse matrix")

Load data into sparse matrix


In [10]:
dtrain = xgb.DMatrix(data=Xtrain_c, missing = NA, label= Ytrain_c)
deval  = xgb.DMatrix(data=Xeval, missing = NA, label = Yeval)
print("Specifying the parameters ... ")

Specifying the parameters ... 


In [11]:
param = {'max_depth': 12,
         'eta': 0.02,
         'subsample': 0.7,
         'colsample_bytree': 0.8,
         'silent': 0,
         'eval_metric': 'auc',
         'alpha': 0,
         'lambda': 1,
         'nthread': 8,
         'objective': 'binary:logistic'}

Set the validations set to watch performance 

In [12]:
watchlist = [(deval, 'eval'), (dtrain, 'train')]
num_round = 510
print("Training ... ")

Training ... 


In [13]:
bst = xgb.train(param, dtrain, num_round, watchlist)

[0]	eval-auc:0.802269	train-auc:0.827405
[1]	eval-auc:0.809492	train-auc:0.835327
[2]	eval-auc:0.812261	train-auc:0.838463
[3]	eval-auc:0.813512	train-auc:0.839555
[4]	eval-auc:0.814739	train-auc:0.840630
[5]	eval-auc:0.815981	train-auc:0.842728
[6]	eval-auc:0.816512	train-auc:0.843284
[7]	eval-auc:0.816645	train-auc:0.843980
[8]	eval-auc:0.817006	train-auc:0.844120
[9]	eval-auc:0.817328	train-auc:0.844013
[10]	eval-auc:0.817643	train-auc:0.844551
[11]	eval-auc:0.817906	train-auc:0.845552
[12]	eval-auc:0.818138	train-auc:0.846342
[13]	eval-auc:0.818407	train-auc:0.846722
[14]	eval-auc:0.818616	train-auc:0.846811
[15]	eval-auc:0.818596	train-auc:0.846873
[16]	eval-auc:0.818723	train-auc:0.847391
[17]	eval-auc:0.818756	train-auc:0.847749
[18]	eval-auc:0.818880	train-auc:0.847970
[19]	eval-auc:0.818961	train-auc:0.848588
[20]	eval-auc:0.818986	train-auc:0.848838
[21]	eval-auc:0.819096	train-auc:0.849111
[22]	eval-auc:0.819096	train-auc:0.849266
[23]	eval-auc:0.819173	train-auc:0.849556
[2

Save models and data

In [ ]:
bst.save_model('./models/xgb_acc.model')
bst.dump_model('./models/xgb_raw_acc.txt')

Load test data 

In [ ]:
Xtest = pd.read_csv("./test/accident_test_ext.csv", index_col=0)
ID= Xtest['ID'].astype(np.int64)
Xtest.drop('ID', axis= 1, inplace=True)
print("Load test samples")

In [ ]:
dtest= xgb.DMatrix(data=Xtest.values, missing = NA)

In [ ]:
print("Prediction")
preds = bst.predict(dtest)

In [ ]:
dtrain.save_binary('./models/binary/dtrain.buffer')
dtest.save_binary('./models/binary/dtest.buffer')
deval.save_binary('./models/binary/deval.buffer')

In [ ]:
submit = pd.DataFrame(data={'ID': ID, 'DRUNK_DR': preds})
submit.to_csv('fars_submit.csv', index = False)